##  Save and load models (https://www.tensorflow.org/tutorials/keras/save_and_load)

Model progress can be saved during and after training. This means a model can resume where it left off and avoid long training times. Saving also means you can share your model and others can recreate your work. When publishing research models and techniques, most machine learning practitioners share:

<ul>
    <li>code to create the model, and</li>
    <li>the trained weights, or parameters, for the model</li>
</ul>

Sharing this data helps others understand how the model works and try it themselves with new data.

<font color="red"><b>Caution:</b> TensorFlow models are code and it is important to be careful with untrusted code. See Using TensorFlow Securely for details.</font>

#### Options

There are different ways to save TensorFlow models depending on the API you're using. This guide uses <a href="https://www.tensorflow.org/guide/keras">tf.keras</a>—a high-level API to build and train models in TensorFlow. The new, high-level <code>.keras</code> format used in this tutorial is recommended for saving Keras objects, as it provides robust, efficient name-based saving that is often easier to debug than low-level or legacy formats. For more advanced saving or serialization workflows, especially those involving custom objects, please refer to the <a href="https://www.tensorflow.org/guide/keras/save_and_serialize">Save and load Keras models guide</a>. For other approaches, refer to the <a href="https://www.tensorflow.org/guide/saved_model">Using the SavedModel format guide</a>.

### Setup

#### Installs and imports

Install and import TensorFlow and dependencies:

In [1]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

In [2]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2025-03-26 03:16:54.173084: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742959014.188773  265335 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742959014.193738  265335 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742959014.204947  265335 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742959014.204965  265335 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742959014.204966  265335 computation_placer.cc:177] computation placer alr

2.19.0


### Get an example dataset

To demonstrate how to save and load weights, you'll use the <a href="http://yann.lecun.com/exdb/mnist/">MNIST</a> dataset. To speed up these runs, use the first 1000 examples:

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

### Define a model

Start by building a simple sequential model:

In [4]:
# Define a simple sequential model
def create_model():
  model = tf.keras.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-03-26 03:40:07.548890: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 407,050 (1.55 MB)

 Trainable params: 407,050 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

### Save checkpoints during training

You can use a trained model without having to retrain it, or pick-up training where you left off in case the training process was interrupted. The <a href="https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint">tf.keras.callbacks.ModelCheckpoint</a> callback allows you to continually save the model both during and at the end of training.

### Checkpoint callback usage

Create a <a href="https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint">tf.keras.callbacks.ModelCheckpoint</a> callback that saves weights only during training:

In [6]:
#checkpoint_path = "training_1/cp.ckpt"
checkpoint_path = "training_1/cp.weights.h5"

checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Epoch 1/10
18/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.8222 - sparse_categorical_accuracy: 0.4015   
Epoch 1: saving model to training_1/cp.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 1.5614 - sparse_categorical_accuracy: 0.5104 - val_loss: 0.7169 - val_sparse_categorical_accuracy: 0.7720
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4584 - sparse_categorical_accuracy: 0.8708
Epoch 2: saving model to training_1/cp.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.4571 - sparse_categorical_accuracy: 0.8712 - val_loss: 0.5259 - val_sparse_categorical_accuracy: 0.8380
Epoch 3/10
17/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2446 - sparse_categorical_accuracy: 0.9411 
Epoch 3: saving model to training_1/cp.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.2611 - sparse_categorical_accuracy: 0.9353 - val_loss: 0.4535 - val_sparse_categorical_accuracy: 0.8560
Epoch 4/10
18/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1718 - sparse_cate

This creates a single collection of TensorFlow checkpoint files that are updated at the end of each epoch:

In [7]:
os.listdir(checkpoint_dir)

['cp.weights.h5']

As long as two models share the same architecture you can share weights between them. So, when restoring a model from weights-only, create a model with the same architecture as the original model and then set its weights.

Now rebuild a fresh, untrained model and evaluate it on the test set. An untrained model will perform at chance levels (~10% accuracy):

In [8]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - 3ms/step - loss: 2.3852 - sparse_categorical_accuracy: 0.1200
Untrained model, accuracy: 12.00%


Then load the weights from the checkpoint and re-evaluate:

In [9]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - 1ms/step - loss: 0.3958 - sparse_categorical_accuracy: 0.8700
Restored model, accuracy: 87.00%


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


### Checkpoint callback options

The callback provides several options to provide unique names for checkpoints and adjust the checkpointing frequency.

Train a new model, and save uniquely named checkpoints once every five epochs:

In [ ]:
!mkdir training_2

In [12]:
# Include the epoch in the file name (uses `str.format`)
#checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_path = "training_2/cp-{epoch:04d}.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Calculate the number of batches per epoch
import math
n_batches = len(train_images) / batch_size
n_batches = math.ceil(n_batches)    # round up the number of batches to the nearest whole integer

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*n_batches)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          batch_size=batch_size, 
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 5: saving model to training_2/cp-0005.weights.h5

Epoch 10: saving model to training_2/cp-0010.weights.h5

Epoch 15: saving model to training_2/cp-0015.weights.h5

Epoch 20: saving model to training_2/cp-0020.weights.h5

Epoch 25: saving model to training_2/cp-0025.weights.h5

Epoch 30: saving model to training_2/cp-0030.weights.h5

Epoch 35: saving model to training_2/cp-0035.weights.h5

Epoch 40: saving model to training_2/cp-0040.weights.h5

Epoch 45: saving model to training_2/cp-0045.weights.h5

Epoch 50: saving model to training_2/cp-0050.weights.h5


Now, review the resulting checkpoints and choose the latest one:

In [13]:
os.listdir(checkpoint_dir)

['cp-0025.weights.h5',
 'cp-0040.weights.h5',
 'cp-0020.weights.h5',
 'cp-0050.weights.h5',
 'cp-0045.weights.h5',
 'cp-0015.weights.h5',
 'cp-0005.weights.h5',
 'cp-0000.weights.h5',
 'cp-0035.weights.h5',
 'cp-0010.weights.h5',
 'cp-0030.weights.h5']

In [18]:
! pwd

/tf/tensor.org/tutorials


In [21]:
os.listdir('/tf/tensor.org/tutorials/training_2')

['cp-0025.weights.h5',
 'cp-0040.weights.h5',
 'cp-0020.weights.h5',
 'cp-0050.weights.h5',
 'cp-0045.weights.h5',
 'cp-0015.weights.h5',
 'cp-0005.weights.h5',
 'cp-0000.weights.h5',
 'cp-0035.weights.h5',
 'cp-0010.weights.h5',
 'cp-0030.weights.h5']

In [28]:
# print(f"{checkpoint_dir} : {tf.train.latest_checkpoint(checkpoint_dir)}")
# print(f"{'/tf/tensor.org/tutorials/training_2'} : {tf.train.latest_checkpoint('/tf/tensor.org/tutorials/training_2')}")
# print(tf.train.latest_checkpoint(os.path.dirname('training_2/cp-0030.weights.h5')))
# print(tf.train.latest_checkpoint('/tf/tensor.org/tutorials/training_2/cp-0030.weights.h5'))

# latest = tf.train.latest_checkpoint(checkpoint_dir)
# latest

# # Create a new model instance
# model = create_model()

# # Load the previously saved weights
# model.load_weights('/tf/tensor.org/tutorials/training_2/cp-0030.weights.h5')

To test, reset the model, and load the latest checkpoint:

In [29]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
# model.load_weights(latest)
# The tf.train.latest_checkpoint doesn't seem to EVER FUCKING WORK
model.load_weights('/tf/tensor.org/tutorials/training_2/cp-0030.weights.h5')

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - 3ms/step - loss: 0.4473 - sparse_categorical_accuracy: 0.8750
Restored model, accuracy: 87.50%


### What are these files?

The above code stores the weights to a collection of <a href="https://www.tensorflow.org/guide/checkpoint">checkpoint</a>-formatted files that contain only the trained weights in a binary format. Checkpoints contain:

<ul>
    <li>One or more shards that contain your model's weights.</li>
    <li>An index file that indicates which weights are stored in which shard.</li>
</ul>

If you are training a model on a single machine, you'll have one shard with the suffix: .data-00000-of-00001

### Manually save weights

To save weights manually, use <code><a href="https://www.tensorflow.org/api_docs/python/tf/keras/Model#save_weights">tf.keras.Model.save_weights</a></code>. By default, <code><a href="https://www.tensorflow.org/api_docs/python/tf/keras">tf.keras</a></code>—and the <code><a href="https://www.tensorflow.org/api_docs/python/tf/keras/Model#save_weights">Model.save_weights</a></code> method in particular—uses the TensorFlow <a href="https://www.tensorflow.org/guide/checkpoint">Checkpoint</a> format with a .ckpt extension. To save in the HDF5 format with a .h5 extension, refer to the <a href="https://www.tensorflow.org/guide/keras/save_and_serialize">Save and load models</a> guide.

In [32]:
! mkdir ./checkpoints/my_checkpoint

In [35]:
# Save the weights
#model.save_weights('./checkpoints/my_checkpoint')
model.save_weights('./checkpoints/my_checkpoint.weights.h5')


# Create a new model instance
model = create_model()

# Restore the weights
#model.load_weights('./checkpoints/my_checkpoint')
model.load_weights('./checkpoints/my_checkpoint.weights.h5')

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - 3ms/step - loss: 2.3212 - sparse_categorical_accuracy: 0.0960
Restored model, accuracy:  9.60%


### Save the entire model

Call <code><a href="https://www.tensorflow.org/api_docs/python/tf/keras/Model#save">tf.keras.Model.save</a></code> to save a model's architecture, weights, and training configuration in a single model.keras zip archive.

An entire model can be saved in three different file formats (the new <code>.keras</code> format and two legacy formats: <code>SavedModel</code>, and <code>HDF5</code>). Saving a model as <code>path/to/model.keras</code> automatically saves in the latest format.

You can switch to the SavedModel format by:

<ul>
    <li>Passing <code>save_format='tf'</code> to <code>save()</code></li>
    <li>Passing a filename without an extension</li>
</ul>

You can switch to the H5 format by:

<ul>
    <li>Passing <code>save_format='h5'</code> to <code>save()</code></li>
    <li>Passing a filename that ends in <code>.h5</code></li>
</ul>

Saving a fully-functional model is very useful—you can load them in TensorFlow.js (<a href="https://www.tensorflow.org/js/tutorials/conversion/import_saved_model">Saved Model</a>, <a href="https://www.tensorflow.org/js/tutorials/conversion/import_keras">HDF5</a>) and then train and run them in web browsers, or convert them to run on mobile devices using TensorFlow Lite (<a href="https://www.tensorflow.org/lite/models/convert/#convert_a_savedmodel_recommended_">Saved Model</a>, <a href="https://www.tensorflow.org/lite/models/convert/#convert_a_keras_model_">HDF5</a>)

*Custom objects (for example, subclassed models or layers) require special attention when saving and loading. Refer to the <b>Saving custom objects</b> section below.

### New high-level .keras format

The new Keras v3 saving format, marked by the <code>.keras</code> extension, is a more simple, efficient format that implements name-based saving, ensuring what you load is exactly what you saved, from Python's perspective. This makes debugging much easier, and it is the recommended format for Keras.

The section below illustrates how to save and restore the model in the <code>.keras</code> format.

In [36]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a `.keras` zip archive.
model.save('my_model.keras')

Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.5533 - sparse_categorical_accuracy: 0.5439 
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4592 - sparse_categorical_accuracy: 0.8806
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2592 - sparse_categorical_accuracy: 0.9387
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2225 - sparse_categorical_accuracy: 0.9443
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1598 - sparse_categorical_accuracy: 0.9589


Reload a fresh Keras model from the <code>.keras</code> zip archive:

In [37]:
new_model = tf.keras.models.load_model('my_model.keras')

# Show the model architecture
new_model.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_18 (Dense)                │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,221,152 (4.66 MB)

 Trainable params: 407,050 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 814,102 (3.11 MB)

Try running evaluate and predict with the loaded model:

In [39]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - 3ms/step - loss: 0.4271 - sparse_categorical_accuracy: 0.8590
Restored model, accuracy: 85.90%
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
(1000, 10)


### SavedModel format

The SavedModel format is another way to serialize models. Models saved in this format can be restored using <code><a href="https://www.tensorflow.org/api_docs/python/tf/keras/models/load_model">tf.keras.models.load_model</a></code> and are compatible with TensorFlow Serving. <a href="https://www.tensorflow.org/guide/saved_model">The SavedModel</a> guide goes into detail about how to <code>serve/inspect</code> the SavedModel. The section below illustrates the steps to save and restore the model.

In [41]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
#model.save('saved_model/my_model')
model.save('saved_model/my_model.keras')

Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.5592 - sparse_categorical_accuracy: 0.5340 
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4488 - sparse_categorical_accuracy: 0.8668
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2632 - sparse_categorical_accuracy: 0.9352
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2011 - sparse_categorical_accuracy: 0.9462
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1581 - sparse_categorical_accuracy: 0.9626


The SavedModel format is a directory containing a protobuf binary and a TensorFlow checkpoint. Inspect the saved model directory:

In [43]:
# my_model directory
! ls saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
! ls saved_model/my_model

my_model.keras
ls: cannot access 'saved_model/my_model': No such file or directory


Reload a fresh Keras model from the saved model:

In [44]:
#new_model = tf.keras.models.load_model('saved_model/my_model')
new_model = tf.keras.models.load_model('saved_model/my_model.keras')

# Check its architecture
new_model.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_22 (Dense)                │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,221,152 (4.66 MB)

 Trainable params: 407,050 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 814,102 (3.11 MB)

The restored model is compiled with the same arguments as the original model. Try running evaluate and predict with the loaded model:

In [45]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - 3ms/step - loss: 0.4399 - sparse_categorical_accuracy: 0.8670
Restored model, accuracy: 86.70%
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
(1000, 10)


### HDF5 format

Keras provides a basic legacy high-level save format using the <a href="https://en.wikipedia.org/wiki/Hierarchical_Data_Format">HDF5</a> standard. 

In [46]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5')

Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.6407 - sparse_categorical_accuracy: 0.4810  
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4370 - sparse_categorical_accuracy: 0.8722
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3238 - sparse_categorical_accuracy: 0.9071
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2393 - sparse_categorical_accuracy: 0.9506
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1454 - sparse_categorical_accuracy: 0.9777


Now, recreate the model from that file:

In [47]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_24 (Dense)                │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 407,052 (1.55 MB)

 Trainable params: 407,050 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

Check its accuracy:

In [48]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 0s - 3ms/step - loss: 0.4364 - sparse_categorical_accuracy: 0.8610
Restored model, accuracy: 86.10%


Keras saves models by inspecting their architectures. This technique saves everything:

<ul>
    <li>The weight values</li>
    <li>The model's architecture</li>
    <li>The model's training configuration (what you pass to the <code>.compile()</code> method)</li>
    <li>The optimizer and its state, if any (this enables you to restart training where you left off)</li>
</ul>

Keras is not able to save the <code>v1.x optimizers</code> (from <code><a href="https://www.tensorflow.org/api_docs/python/tf/compat/v1/train">tf.compat.v1.train</a></code>) since they aren't compatible with checkpoints. For v1.x optimizers, you need to re-compile the model after loading—losing the state of the optimizer.

### Saving custom objects

If you are using the SavedModel format, you can skip this section. The key difference between high-level <code>.keras</code>/HDF5 formats and the low-level SavedModel format is that the <code>.keras</code>/HDF5 formats uses object configs to save the model architecture, while SavedModel saves the execution graph. Thus, SavedModels are able to save custom objects like subclassed models and custom layers without requiring the original code. However, debugging low-level SavedModels can be more difficult as a result, and we recommend using the high-level <code>.keras</code> format instead due to its name-based, Keras-native nature.

To save custom objects to <code>.keras</code> and HDF5, you must do the following:

<ol>
    <li>Define a <code>get_config</code> method in your object, and optionally a <code>from_config</code> classmethod.
        <ul>
            <li><code>get_config(self)</code> returns a JSON-serializable dictionary of parameters needed to recreate the object.</li>
            <li><code>from_config(cls, config)</code> uses the returned config from <code>get_config</code> to create a new object. By default, this function will use the config as initialization kwargs (<code>return cls(**config))</code>.</li>
        <ul>
    </li>
    <li>Pass the custom objects to the model in one of three ways:
        <ul>
            <li>Register the custom object with the <code><a href="https://www.tensorflow.org/api_docs/python/tf/keras/utils/register_keras_serializable">@tf.keras.utils.register_keras_serializable</a></code> decorator. <b>(recommended)</b></li>
            <li>Directly pass the object to the <code>custom_objects</code> argument when loading the model. The argument must be a dictionary mapping the string class name to the Python class. E.g., <code>tf.keras.models.load_model(path, custom_objects={'CustomLayer': CustomLayer})</code></li>
            <li>Use a <code><a href="https://www.tensorflow.org/api_docs/python/tf/keras/utils/CustomObjectScope">tf.keras.utils.custom_object_scope</a></code> with the object included in the <code>custom_objects</code> dictionary argument, and place a <code><a href="https://www.tensorflow.org/api_docs/python/tf/keras/models/load_model">tf.keras.models.load_model(path)</a></code> call within the scope.</li>
        </ul>
    </li>
</ol>

Refer to the <a href="https://www.tensorflow.org/guide/keras/custom_layers_and_models">Writing layers and models from scratch</a> tutorial for examples of custom objects and get_config.